<div class="alert alert-block alert-info">
<b><h2><center>Data Science Project - Google Maps Nearby Places</center></h2></b>


Obtaining data using Google Maps API</div>

By Eduardo Teixeira de Aguiar Veiga

Contact: etaveiga@gmail.com
<nav>
<a href="https://www.linkedin.com/in/eduardo-veiga-0728221a6/">LinkedIn</a> |
<a href="https://github.com/eduardotaveiga">Github</a> |
    <a href="https://medium.com/@etaveiga">Medium</a>
</nav>

If you have any suggestions (including english tips!) feel free to send me a message!

![](https://images.unsplash.com/photo-1604357209793-fca5dca89f97?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=464&q=80)

*Image source: https://unsplash.com/pt-br/fotografias/rEn-AdBr3Ig*

<div class="alert alert-block alert-info">
<b><h2><center>Project Idea</center></h2></b></div>


Google Maps is a well -nown application for calculating routes, finding places of interest, and visualizing locations. For us, programmers and data scientists, it has an additional functionality that some may find really useful: we can extract data. Actually, you can also use Google Maps' functions to create your own delivery app for routes or embed static/dynamic maps in your web application. However, in this project, I'll show you how to use the Maps API to obtain data about places around you.

Google Maps Places API documentation: https://developers.google.com/maps/documentation/places/web-service

An important thing to keep in mind is that the Places API is not free, but Google Maps Platform provides a U$200 credit at the beginning of each month for us to try out its functionalities.

In [1]:
import googlemaps  # pip install googlemaps
import pandas as pd
import numpy as np
import re
from time import sleep

# This one will be used later, see https://jupyter-gmaps.readthedocs.io/en/latest/install.html for instalation
import gmaps 

<div class="alert alert-block alert-info">
<b><h2><center>Nearby Search</center></h2></b></div>


First, you need to enable an API key for the Places API in the Google Cloud Console. Then, create a Client object that contains access to all the APIs you want to use.

In [2]:
API_KEY = 'YOUR_API_KEY'
map_client = googlemaps.Client(API_KEY)

We are going to use the Nearby Search function. By providing a location (latitude, longitude) and a radius (in meters), it will return 20 places inside a circle centered on that location (with the specified radius). If this circle is large enough, a *next_page_token* will be provided, which we can use to request an additional 20 entries (passing this token as an argument). Note that the maximum number of places that Google Maps allows us to get is 60. If you need to obtain more places, make sure to add a little delay between each query, otherwise you will raise an error.

Also, we can refine our search by specifying a type (such as 'restaurant' or 'drugstore') and keywords (such as 'pizza' and a particular city). Here, I'm searching for restaurants that sell hamburgers near the center of São Paulo, Brazil.

In [3]:
location = (-23.54158115282599, -46.63009071740232)  # center location
radius = 50000  # radius in meters
business = 'restaurant'
keyword = 'hamburger'

# requesting data
response = map_client.places_nearby(location=location, type=business, radius=radius, keyword=keyword)

# appending our data to a list
results = []
results.extend(response['results'])

# if there is a next_page, get the data from it too (up to 60)
next_page = response['next_page_token']
while True:
    try:
        sleep(2)
        response = map_client.places_nearby(location=location, type=business, radius=radius, keyword=keyword, page_token=next_page)
        results.extend(response['results'])
        next_page = response['next_page_token']
    except:
        break

Since we are limited to 60 results, Google Maps chooses which places to provide using an internal metric called 'prominence.' This metric evaluates how important a business is based on factors such as the place's ranking in Google's index, global popularity, and other factors. You can change this behavior by setting the 'rankby' argument to 'distance' and removing the radius. With this configuration, the function will return the 60 nearest places to the center location.

```python
map_client.places_nearby(location=location, type=business, rankby='distance', keyword=keyword)
```

Finally, we put those results in a pandas DataFrame

In [4]:
df = pd.DataFrame(results)
df.head()

,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,opening_hours,photos,place_id,plus_code,price_level,rating,reference,scope,types,user_ratings_total,vicinity
0,OPERATIONAL,"{'location': {'lat': -23.5355551, 'lng': -46.6...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Zé do Hamburger,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJX4fH0h1YzpQRNiQcO6OTu3I,"{'compound_code': 'F87J+QP Perdizes, São Paulo...",3.0,4.5,ChIJX4fH0h1YzpQRNiQcO6OTu3I,GOOGLE,"[meal_takeaway, restaurant, point_of_interest,...",8429,"R. Itapicuru, 419 - Perdizes, São Paulo"
1,OPERATIONAL,"{'location': {'lat': -23.5398466, 'lng': -46.5...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,It Burger,{'open_now': True},"[{'height': 4160, 'html_attributions': ['<a hr...",ChIJAQQ_velezpQRku2Ag6G61DY,"{'compound_code': 'FC6H+25 São Paulo, State of...",2.0,4.5,ChIJAQQ_velezpQRku2Ag6G61DY,GOOGLE,"[meal_takeaway, restaurant, point_of_interest,...",5336,"R. Fernandes Pinheiro, 51 - Tatuapé, São Paulo"
2,OPERATIONAL,"{'location': {'lat': -23.6072709, 'lng': -46.6...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Chicohamburger,{'open_now': True},"[{'height': 1918, 'html_attributions': ['<a hr...",ChIJYwp9cBtazpQRu26Yrl02A90,"{'compound_code': '98VM+3X São Paulo, State of...",2.0,4.5,ChIJYwp9cBtazpQRu26Yrl02A90,GOOGLE,"[restaurant, point_of_interest, food, establis...",10795,"Av. Ibirapuera, 2713 - Moema, São Paulo"
3,OPERATIONAL,"{'location': {'lat': -23.5662196, 'lng': -46.6...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,"4 Eat Burger Bar -Restaurante , Hamburgueria e...",{'open_now': True},"[{'height': 720, 'html_attributions': ['<a hre...",ChIJi1gGwqZXzpQRh4odF6YJP-g,"{'compound_code': 'C8M3+FM Pinheiros, São Paul...",2.0,4.1,ChIJi1gGwqZXzpQRh4odF6YJP-g,GOOGLE,"[meal_takeaway, restaurant, point_of_interest,...",424,"R. Guaicuí, 29 - Pinheiros, São Paulo"
4,OPERATIONAL,"{'location': {'lat': -23.5838134, 'lng': -46.6...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,New Dog Hamburger,{'open_now': True},"[{'height': 1440, 'html_attributions': ['<a hr...",ChIJWRNvGF9XzpQRetBgux1t6qw,"{'compound_code': 'C88G+FX São Paulo, State of...",3.0,4.3,ChIJWRNvGF9XzpQRetBgux1t6qw,GOOGLE,"[meal_takeaway, restaurant, point_of_interest,...",8391,"R. Joaquim Floriano, 254 - Itaim Bibi, São Paulo"


The DataFrame makes it easier to modify the data and facilitates access to specific pieces of information, such as latitude and longitude. The next cell does the following:

- Create a new column with the URL that leads to the location of each place on Google Maps. 
- Select only business that are operating  
- Extract the latitudes and longitudes from the geometry column and drop it  
- Drop two columns related to icon images  
- Drop the *opening_hours* column since it does not provide useful information  
- Try to extract both neighborhood and city  
- Drop the reference column (same as place_id)  
- Drop the scope column (it is useless)  

In [5]:
df['url'] = 'https://www.google.com/maps/place/?q=place_id:'+df['place_id']
df = df[df['business_status'] == 'OPERATIONAL']

def return_lat(item):
    return item['location']['lat']

def return_lon(item):
    return item['location']['lng']

df['lat'] = df['geometry'].apply(return_lat)
df['lon'] = df['geometry'].apply(return_lon)
df = df.drop('geometry', axis='columns')

df = df.drop('icon_background_color', axis='columns')
df = df.drop('icon_mask_base_uri', axis='columns')
df = df.drop('opening_hours', axis='columns')

def get_neighborhood(item):
    try:
        result_str = item['compound_code']
        regex_result = re.findall(r'([.* ])(.+?),', result_str)
        neighb = regex_result[0][1]
    except:
        return np.nan
    else:
        return neighb
    
    
def get_city(item):
    try:
        result_str = item['compound_code']
        regex_result = re.findall(r'([.* ])(.+?),(.*)[-|,]', result_str)
        city = regex_result[0][2]
    except:
        return np.nan
    else:
        return city

df['neighborhood'] = df['plus_code'].apply(get_neighborhood)
df['city'] = df['plus_code'].apply(get_city)

df = df.drop('reference', axis='columns')
df = df.drop('scope', axis='columns')

df.head()

,business_status,icon,name,photos,place_id,plus_code,price_level,rating,types,user_ratings_total,vicinity,url,lat,lon,neighborhood,city
0,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,Zé do Hamburger,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJX4fH0h1YzpQRNiQcO6OTu3I,"{'compound_code': 'F87J+QP Perdizes, São Paulo...",3.0,4.5,"[meal_takeaway, restaurant, point_of_interest,...",8429,"R. Itapicuru, 419 - Perdizes, São Paulo",https://www.google.com/maps/place/?q=place_id:...,-23.535555,-46.668217,Perdizes,São Paulo
1,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,It Burger,"[{'height': 4160, 'html_attributions': ['<a hr...",ChIJAQQ_velezpQRku2Ag6G61DY,"{'compound_code': 'FC6H+25 São Paulo, State of...",2.0,4.5,"[meal_takeaway, restaurant, point_of_interest,...",5336,"R. Fernandes Pinheiro, 51 - Tatuapé, São Paulo",https://www.google.com/maps/place/?q=place_id:...,-23.539847,-46.572026,São Paulo,NaN
2,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,Chicohamburger,"[{'height': 1918, 'html_attributions': ['<a hr...",ChIJYwp9cBtazpQRu26Yrl02A90,"{'compound_code': '98VM+3X São Paulo, State of...",2.0,4.5,"[restaurant, point_of_interest, food, establis...",10795,"Av. Ibirapuera, 2713 - Moema, São Paulo",https://www.google.com/maps/place/?q=place_id:...,-23.607271,-46.665124,São Paulo,NaN
3,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,"4 Eat Burger Bar -Restaurante , Hamburgueria e...","[{'height': 720, 'html_attributions': ['<a hre...",ChIJi1gGwqZXzpQRh4odF6YJP-g,"{'compound_code': 'C8M3+FM Pinheiros, São Paul...",2.0,4.1,"[meal_takeaway, restaurant, point_of_interest,...",424,"R. Guaicuí, 29 - Pinheiros, São Paulo",https://www.google.com/maps/place/?q=place_id:...,-23.566220,-46.695841,Pinheiros,São Paulo
4,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,New Dog Hamburger,"[{'height': 1440, 'html_attributions': ['<a hr...",ChIJWRNvGF9XzpQRetBgux1t6qw,"{'compound_code': 'C88G+FX São Paulo, State of...",3.0,4.3,"[meal_takeaway, restaurant, point_of_interest,...",8391,"R. Joaquim Floriano, 254 - Itaim Bibi, São Paulo",https://www.google.com/maps/place/?q=place_id:...,-23.583813,-46.672560,São Paulo,NaN


<div class="alert alert-block alert-info">
<b><h2><center>Place Search</center></h2></b></div>


Now, we can extract more data about each place individually. To do so, we will use the Place Search functionality. The code is basically a bunch of try/except statements to ensure that we will get the data or return a Nan, appending it to a list.

Something important to mention is that not all data is available for all places. For example, some places may have a serves_beer field, while others won't. To see a full list of possible data returned, check out the documentation.

For the reviews section, we are limited to extract only 5 reviews per place.

In [6]:
# Some lists to store the data we want
street = []
neighborhood = []
city = []
week_days = [[], [], [], [], [], [], []]
editorial_reviews = []
phones = []
total_reviews = []
websites = []
author_names = [[], [], [], [], []]
ratings = [[], [], [], [], []]
relative_times = [[], [], [], [], []]
texts = [[], [], [], [], []]
references = []

# string manipulation (useful for the current_opening_hours data section)
def replace_unicode(string):
    return string.replace('\u2009', ' ').replace('\u202f', ' ')

def decode_time(list_of_times):
    new_time = list(map(replace_unicode, list_of_times))
    return new_time

# For each place id, try to extract a particular information
for p_id in df['place_id']:
    place = map_client.place(p_id)
    street.append(place['result']['address_components'][1]['long_name'])
    neighborhood.append(place['result']['address_components'][2]['long_name'])
    city.append(place['result']['address_components'][3]['long_name'])
    
    try:
        hour_list = place['result']['current_opening_hours']['weekday_text']
    except:
        week_days[0].append(np.nan)
        week_days[1].append(np.nan)
        week_days[2].append(np.nan)
        week_days[3].append(np.nan)
        week_days[4].append(np.nan)
        week_days[5].append(np.nan)
        week_days[6].append(np.nan)
    else:
        hourly = decode_time(hour_list)
        idx = 0
        for c in hourly:
            pattern = '(\w*:)(.*)'
            regex = re.findall(pattern, c)
            week_days[idx].append(regex[0][1].strip())
            idx += 1
    
    try:
        editorial = place['result']['editorial_summary']['overview']
    except:
        editorial_reviews.append(np.nan)
    else:
        editorial_reviews.append(editorial)
    
    try:
        phone = place['result']['formatted_phone_number']
    except:
        phones.append(np.nan)
    else:
        phones.append(phone)
    
    try:
        t_r = place['result']['user_ratings_total']
    except:
        total_reviews.append(np.nan)
    else:
        total_reviews.append(t_r)
    
    try:
        website = place['result']['website']
    except:
        websites.append(np.nan)
    else:
        websites.append(website)
    
    # If the reviews section does not exist, append Nan
    try:
        reviews = place['result']['reviews']
    except:
        idx = 0
        while idx+1 < 5:
            author_names[idx].append(np.nan)
            ratings[idx].append(np.nan)
            relative_times[idx].append(np.nan)
            texts[idx].append(np.nan)
            idx += 1
    else:
        idx = 0
        for c in place['result']['reviews']:
            author_names[idx].append(c['author_name'])
            ratings[idx].append(c['rating'])
            relative_times[idx].append(c['relative_time_description'])
            texts[idx].append(c['text'])
            idx += 1

        while idx < 5:  # if there is less than 5 reviews, append nan to the remaining
            author_names[idx].append(np.nan)
            ratings[idx].append(np.nan)
            relative_times[idx].append(np.nan)
            texts[idx].append(np.nan)
            idx += 1
    
    references.append(place['result']['reference'])

Insert all these data into a pandas DataFrame

In [8]:
data = {'street':street, 'neighborhood':neighborhood, 'city':city, 
        'opening_hours_monday':week_days[0],'opening_hours_tuesday':week_days[1],'opening_hours_wednesday':week_days[2],'opening_hours_thursday':week_days[3],'opening_hours_friday':week_days[4],'opening_hours_saturday':week_days[5],'opening_hours_sunday':week_days[6],
       'editorial_review':editorial_reviews, 'phone':phones, 'website':websites, 
        'review_1':texts[0],'review_2':texts[1],'review_3':texts[2],'review_4':texts[3],'review_5':texts[4],
        'rating_1':ratings[0],'rating_2':ratings[1],'rating_3':ratings[2],'rating_4':ratings[3],'rating_5':ratings[4],
        'author_1':author_names[0],'author_2':author_names[1],'author_3':author_names[2],'author_4':author_names[3],'author_5':author_names[4],
        'relative_time_1':relative_times[0],'relative_time_2':relative_times[1],'relative_time_3':relative_times[2],'relative_time_4':relative_times[3],'relative_time_5':relative_times[4],'total_reviews':total_reviews,
        'reference':references}

place_search_df = pd.DataFrame(data=data)
place_search_df.head()

,street,neighborhood,city,opening_hours_monday,opening_hours_tuesday,opening_hours_wednesday,opening_hours_thursday,opening_hours_friday,opening_hours_saturday,opening_hours_sunday,...,author_3,author_4,author_5,relative_time_1,relative_time_2,relative_time_3,relative_time_4,relative_time_5,total_reviews,reference
0,Rua Itapicuru,Perdizes,São Paulo,12:00 – 11:00 PM,12:00 – 11:00 PM,12:00 – 11:00 PM,12:00 – 11:00 PM,12:00 PM – 12:00 AM,12:00 PM – 12:00 AM,12:00 – 11:00 PM,...,Daniel Inhauser Mesojedovas,Rogério Brandão,Christian Barone,a year ago,2 years ago,3 years ago,6 months ago,3 months ago,8429,ChIJX4fH0h1YzpQRNiQcO6OTu3I
1,Rua Fernandes Pinheiro,Tatuapé,São Paulo,11:40 AM – 11:00 PM,11:40 AM – 11:00 PM,11:40 AM – 11:00 PM,11:40 AM – 11:00 PM,11:40 AM – 1:00 AM,11:40 AM – 1:00 AM,11:40 AM – 11:00 PM,...,Hugo Yamashita,Rafael Estevam,Angela Renshaw,4 years ago,7 months ago,4 years ago,5 years ago,4 years ago,5336,ChIJAQQ_velezpQRku2Ag6G61DY
2,Avenida Ibirapuera,Moema,São Paulo,11:00 AM – 4:00 AM,11:00 AM – 4:00 AM,11:00 AM – 4:00 AM,11:00 AM – 5:00 AM,11:00 AM – 6:00 AM,11:00 AM – 6:00 AM,11:00 AM – 4:00 AM,...,Fernanda - Livros by F,Solange Conserva,Lucio Chagas,5 months ago,a month ago,5 years ago,5 months ago,4 years ago,10795,ChIJYwp9cBtazpQRu26Yrl02A90
3,Rua Guaicuí,Pinheiros,São Paulo,12:00 – 10:00 PM,12:00 – 10:00 PM,12:00 – 10:00 PM,12:00 – 10:00 PM,12:00 – 10:00 PM,12:00 – 10:00 PM,12:00 – 10:00 PM,...,Gokhan Koyuncu,Kleber Silva,Pedro Abilleira,6 years ago,5 years ago,4 years ago,6 years ago,3 years ago,424,ChIJi1gGwqZXzpQRh4odF6YJP-g
4,Rua Joaquim Floriano,Itaim Bibi,São Paulo,Open 24 hours,Open 24 hours,Open 24 hours,Open 24 hours,Open 24 hours,Open 24 hours,Open 24 hours,...,Andrés Suárez,Jim H,Sérgio Lisboa Machado,5 months ago,3 months ago,11 months ago,11 months ago,3 years ago,8391,ChIJWRNvGF9XzpQRetBgux1t6qw


And join it with the Nearby Search set created above. You may have noticed that we have two columns for both neighborhood and city. I decided to do keep them because the Nearby Search and Place Search sometimes don't return this information correctly (or it was wrongly extracted because of formatting issues). With both columns, I hope that at least one of them will have returned pertinent values (which can be later checked by an analyst)..

In [3]:
final_df = df.set_index('place_id').join(place_search_df.set_index('reference'), lsuffix='_nearby', rsuffix='_place')
final_df.head()

,business_status,icon,name,photos,plus_code,price_level,rating,types,user_ratings_total,vicinity,...,author_2,author_3,author_4,author_5,relative_time_1,relative_time_2,relative_time_3,relative_time_4,relative_time_5,total_reviews
0,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,Zé do Hamburger,"[{'height': 3024, 'html_attributions': ['<a hr...","{'compound_code': 'F87J+QP Perdizes, São Paulo...",3.0,4.5,"['meal_takeaway', 'restaurant', 'point_of_inte...",8429,"R. Itapicuru, 419 - Perdizes, São Paulo",...,Flavio Gilberti,Daniel Inhauser Mesojedovas,Rogério Brandão,Christian Barone,a year ago,2 years ago,3 years ago,6 months ago,3 months ago,8429
1,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,It Burger,"[{'height': 4160, 'html_attributions': ['<a hr...","{'compound_code': 'FC6H+25 São Paulo, State of...",2.0,4.5,"['meal_takeaway', 'restaurant', 'point_of_inte...",5336,"R. Fernandes Pinheiro, 51 - Tatuapé, São Paulo",...,Mário Reinaldo Honorato,Hugo Yamashita,Rafael Estevam,Angela Renshaw,4 years ago,7 months ago,4 years ago,5 years ago,4 years ago,5336
2,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,Chicohamburger,"[{'height': 1918, 'html_attributions': ['<a hr...","{'compound_code': '98VM+3X São Paulo, State of...",2.0,4.5,"['restaurant', 'point_of_interest', 'food', 'e...",10795,"Av. Ibirapuera, 2713 - Moema, São Paulo",...,Cesar Kuroiwa,Fernanda - Livros by F,Solange Conserva,Lucio Chagas,5 months ago,a month ago,5 years ago,5 months ago,4 years ago,10795
3,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,"4 Eat Burger Bar -Restaurante , Hamburgueria e...","[{'height': 720, 'html_attributions': ['<a hre...","{'compound_code': 'C8M3+FM Pinheiros, São Paul...",2.0,4.1,"['meal_takeaway', 'restaurant', 'point_of_inte...",424,"R. Guaicuí, 29 - Pinheiros, São Paulo",...,Solaiman Shokur,Gokhan Koyuncu,Kleber Silva,Pedro Abilleira,6 years ago,5 years ago,4 years ago,6 years ago,3 years ago,424
4,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,New Dog Hamburger,"[{'height': 1440, 'html_attributions': ['<a hr...","{'compound_code': 'C88G+FX São Paulo, State of...",3.0,4.3,"['meal_takeaway', 'restaurant', 'point_of_inte...",8391,"R. Joaquim Floriano, 254 - Itaim Bibi, São Paulo",...,Lance Noe,Andrés Suárez,Jim H,Sérgio Lisboa Machado,5 months ago,3 months ago,11 months ago,11 months ago,3 years ago,8391


In [11]:
final_df.to_csv('final_df.csv', index=False)
final_df.columns

Index(['business_status', 'icon', 'name', 'photos', 'plus_code', 'price_level',
       'rating', 'types', 'user_ratings_total', 'vicinity', 'url', 'lat',
       'lon', 'neighborhood_nearby', 'city_nearby', 'street',
       'neighborhood_place', 'city_place', 'opening_hours_monday',
       'opening_hours_tuesday', 'opening_hours_wednesday',
       'opening_hours_thursday', 'opening_hours_friday',
       'opening_hours_saturday', 'opening_hours_sunday', 'editorial_review',
       'phone', 'website', 'review_1', 'review_2', 'review_3', 'review_4',
       'review_5', 'rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5',
       'author_1', 'author_2', 'author_3', 'author_4', 'author_5',
       'relative_time_1', 'relative_time_2', 'relative_time_3',
       'relative_time_4', 'relative_time_5', 'total_reviews'],
      dtype='object')

<div class="alert alert-block alert-info">
<b><h2><center>Interactive Map</center></h2></b></div>


There is another interesting Python package called gmaps, which allows us to embed interactive maps inside our Jupyter notebooks. Take a look at the following example:

First, we define a set of locations (latitudes, longitudes) we want to highlight. Here, I'll use the latitudes and longitudes of the places we extracted using Google Maps API.

In [12]:
locations = []
for c in range(len(df)):
    locations.append((final_df['lat'].iloc[c], final_df['lon'].iloc[c]))

Then, we configure a map using gmaps. There is a list of different styles available to plot maps, along with different possible layers to add. I'll add a heatmap layer to get an overview of the distribution of the places we found and a markers layer, with only one marker, to highlight the center location.

In [14]:
gmaps.configure(api_key='place_search_df')

fig = gmaps.figure(map_type='ROADMAP')

heatmap_layer = gmaps.heatmap_layer(locations)

markers = gmaps.marker_layer([location])

fig.add_layer(heatmap_layer)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

We can also modify different parameters of each layer, like the intensity of the heatmap layer and the size (radius) of each data point. Check out the gmaps documentation for more!

In [17]:
heatmap_layer.max_intensity = 2
heatmap_layer.point_radius = 10
heatmap_layer.dissipating  = True